In [ ]:
import pandas as pd
import torch
import re
import ast
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset = load_dataset('saracandu/my_wikihop', split='train')
sources = dataset['supports']

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", use_fast = False)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def produce_prompt(source):
    messages = [ 
        {"role": "system", "content": """
        You are a helpful AI assistant. You are given a long document as context and you are asked to produce a meaningful summary of it.
        Please summarize in less than 500 words.
        """}, 
        {"role": "user", "content": source},
    ] 
    return messages

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 700, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

In [8]:
torch.cuda.empty_cache()

In [9]:
output = pipe(produce_prompt(sources[4]), **generation_args) 
print(output[0]['generated_text']) 

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.97 GiB. GPU 0 has a total capacity of 31.73 GiB of which 310.19 MiB is free. Process 2133144 has 9.85 GiB memory in use. Process 2138557 has 11.47 GiB memory in use. Including non-PyTorch memory, this process has 10.11 GiB memory in use. Of the allocated memory 9.69 GiB is allocated by PyTorch, and 55.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
summ[3]